# Setup

In [1]:
POSTGRESQL_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
# DOCKER_DNS = ["10.15.20.1"]

POSTGRESQL_NAME = "postgres"
# POSTGRESQL_HOSTNAME = f"{POSTGRESQL_NAME}.mavasbel.vpn.itam.mx"
# POSTGRESQL_IP = "10.15.20.2"
POSTGRESQL_PORT = 5423

POSTGRESQL_WORKDIR = "/"
POSTGRESQL_DATADIR = "/var/lib/postgresql"

POSTGRESQL_INIT_USER = "postgres"
POSTGRESQL_INIT_PASSWORD = "password"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
LOCALHOST_DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")

mount_path = Path(LOCALHOST_DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Stop postgresql.docker-compose.yml

In [3]:
!docker compose -f postgresql.docker-compose.yml down -v

 Container postgres  Stopping
 Container postgres  Stopped
 Container postgres  Removing
 Container postgres  Removed
 Network postgres-compose_postgres-cluster  Removing
 Network postgres-compose_postgres-cluster  Removed


In [4]:
import shutil

if POSTGRESQL_START_FROM_SCRATCH:
    shutil.rmtree(LOCALHOST_DOCKER_MOUNTDIR, ignore_errors=True)
    Path(LOCALHOST_DOCKER_MOUNTDIR).mkdir(parents=True, exist_ok=True)

# Start postgresql.docker-compose.yml

In [5]:
import os
import yaml
from IPython.display import Markdown, display

node_cpus = "2.0"
node_memory = "2G"
node_start_heap = "1G"
node_max_heap = "2G"

postgresql_compose_dict = {
    "name": "postgres-compose",
    "services": {},
    "networks": {"postgres-cluster": {"driver": "bridge"}},
}


postgresql_compose_dict["services"][POSTGRESQL_NAME] = {
    "image": "postgres:18.1-trixie",
    "container_name": POSTGRESQL_NAME,
    "environment": {
        "POSTGRES_DB": "postgres",
        "POSTGRES_USER": f"{POSTGRESQL_INIT_USER}",
        "POSTGRES_PASSWORD": f"{POSTGRESQL_INIT_PASSWORD}",
        # "DBS_LIST": "bank_db,ecommerce_db,healthcare_db,social_media_db,streaming_service_db",
    },
    "volumes": [
        f"{os.path.join(LOCALHOST_DOCKER_MOUNTDIR, POSTGRESQL_NAME, "data")}:{POSTGRESQL_DATADIR}",
        f"{os.path.join(LOCALHOST_DOCKER_MOUNTDIR, POSTGRESQL_NAME, "schemas")}:{POSTGRESQL_WORKDIR}/schemas",
        f"{os.path.join(LOCALHOST_WORKDIR, "init-db.sh")}:/docker-entrypoint-initdb.d/init-db.sh:ro",
    ],
    "networks": ["postgres-cluster"],
    # "hostname": f"{POSTGRESQL_HOSTNAME}",
    "ports": [
        f"{POSTGRESQL_PORT}:5432",
    ],
    "extra_hosts": [
        f"{DOCKER_INTERNAL_HOST}:host-gateway",
        # f"{POSTGRESQL_HOSTNAME}:host-gateway",
    ],
    # "dns": DOCKER_DNS,
    "deploy": {"resources": {"limits": {"cpus": node_cpus, "memory": node_memory}}},
    "restart": "unless-stopped",
    "healthcheck": {
        "test": [
            "CMD-SHELL",
            " && ".join(
                [
                    "test -f /tmp/dbs_initialized",
                    f"pg_isready -U {POSTGRESQL_INIT_USER} -d postgres",
                    " ".join(
                        [
                            "for db in $$(echo $$DBS_LIST | tr ',' ' '); do",
                            f'psql -U {POSTGRESQL_INIT_USER} -d $$db -c "SELECT 1" || exit 1;'
                            "done",
                        ]
                    ),
                ]
            ),
        ],
        "interval": "5s",
        "timeout": "10s",
        "retries": 10,
        "start_period": "20s",
    },
}

postgresql_compose_yaml_path = os.path.join(
    LOCALHOST_WORKDIR, "postgresql.docker-compose.yml"
)
postgresql_compose_yaml_contents = yaml.dump(
    postgresql_compose_dict, default_flow_style=False, sort_keys=False, indent=4
)
with open(postgresql_compose_yaml_path, "w") as f:
    f.write(postgresql_compose_yaml_contents)

print(f"Successfully created: '{os.path.relpath(postgresql_compose_yaml_path)}'")
display(Markdown(f"```yaml\n{postgresql_compose_yaml_contents}\n```"))

Successfully created: 'postgresql.docker-compose.yml'


```yaml
name: postgres-compose
services:
    postgres:
        image: postgres:18.1-trixie
        container_name: postgres
        environment:
            POSTGRES_DB: postgres
            POSTGRES_USER: postgres
            POSTGRES_PASSWORD: password
        volumes:
        - .\mount\postgres\data:/var/lib/postgresql
        - .\mount\postgres\schemas://schemas
        - .\init-db.sh:/docker-entrypoint-initdb.d/init-db.sh:ro
        networks:
        - postgres-cluster
        ports:
        - 5423:5432
        extra_hosts:
        - host.docker.internal:host-gateway
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        restart: unless-stopped
        healthcheck:
            test:
            - CMD-SHELL
            - test -f /tmp/dbs_initialized && pg_isready -U postgres -d postgres &&
                for db in $$(echo $$DBS_LIST | tr ',' ' '); do psql -U postgres -d
                $$db -c "SELECT 1" || exit 1;done
            interval: 5s
            timeout: 10s
            retries: 10
            start_period: 20s
networks:
    postgres-cluster:
        driver: bridge

```

In [6]:
!docker compose -f postgresql.docker-compose.yml up -d --wait

 Network postgres-compose_postgres-cluster  Creating
 Network postgres-compose_postgres-cluster  Created
 Container postgres  Creating
 Container postgres  Created
 Container postgres  Starting
 Container postgres  Started
 Container postgres  Waiting
 Container postgres  Healthy
